In [5]:
%pwd

'/Users/ekselan/Desktop/LAMBDA/CS-1/CS1_Build'

In [7]:
import pandas as pd

In [2]:
!which python

/Applications/anaconda3/bin/python


In [3]:
!pip install pandas

In [4]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'